In [ ]:
!wget 'https://drive.google.com/uc?id=1ujxgL5uW1--nSCDoaiK4oaazQot6roav' -O dataset.zip
!unzip dataset.zip

--2024-02-23 12:24:41--  https://drive.google.com/uc?id=1ujxgL5uW1--nSCDoaiK4oaazQot6roav
Resolving drive.google.com (drive.google.com)... 74.125.135.139, 74.125.135.100, 74.125.135.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.135.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ujxgL5uW1--nSCDoaiK4oaazQot6roav [following]
--2024-02-23 12:24:41--  https://drive.usercontent.google.com/download?id=1ujxgL5uW1--nSCDoaiK4oaazQot6roav
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.202.132, 2607:f8b0:400e:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.202.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 982975 (960K) [application/octet-stream]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 959.94K  --.-KB/s    in 0.01s   

2024-02-23 12:24:42 (92.8 MB/s) -

## Back Translation

In [ ]:
!pip install --upgrade google-cloud-translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-translate
    Found existing installation: google-cloud-translate 3.11.3
    Uninstalling google-cloud-translate-3.11.3:
      Successfully uninstalled google-cloud-translate-3.11.3


by setting the variables, we will first translare each sentence of the dataset to the target language, which is set to "fa" standing for Farsi as an example, then we translate it back to the source language, which is set to "en" standing for English, as the dataset is written in English.

### This is an example of the augmented sentence:


original sentence:

West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .

back_translated:

West Indies all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to go top of the county championship.



In [ ]:
import os
from typing import List, Tuple
import random
import requests

def back_translate_sentence(text, target_language, source_language='en') -> str:
    """ Translates text to the target language and back to the source language. """
    print(text)
    print(translate_url)
    print(api_key)

    # Translate to target language
    params_to_target = {
        'key': api_key,
        'q': text,
        'target': target_language,
        'source': source_language
    }
    response_to_target = requests.get(translate_url, params=params_to_target)
    translated_text = response_to_target.json()['data']['translations'][0]['translatedText']

    # Translate back to source language
    params_to_source = {
        'key': api_key,
        'q': translated_text,
        'target': source_language,
        'source': target_language
    }
    response_to_source = requests.get(translate_url, params=params_to_source)
    back_translated_text = response_to_source.json()['data']['translations'][0]['translatedText']

    return back_translated_text

## Synonym Replacement

We use NLTK which provides a WordNet interface that can be used to find synonyms. The candidates to be altered are being chosen in a random manner. Also, The number of altered words can be increased (default is set to 1).

In [ ]:
!pip install nltk

In [ ]:
import random
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')  # Tokenizer models
nltk.download('averaged_perceptron_tagger')  # Part-of-speech tagger
nltk.download('wordnet')  # WordNet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Function to get synonyms
def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

# Function to replace a specified number of random non-entity words with their synonyms in a sentence
def replace_random_words_with_synonyms(sentence, replace_count = 4):
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)

    # Filter out named entities and words without synonyms
    replaceable_words = [(word, tag) for word, tag in tagged_words if 'NNP' not in tag and get_synonyms(word)]

    # Limit the number of replacements to either the specified number or the number of replaceable words, whichever is smaller
    replace_count = min(replace_count, len(replaceable_words))

    altered_sentence = sentence
    for _ in range(replace_count):
        if not replaceable_words:
            break  # Break out of the loop if there are no more replaceable words

        # Randomly select a word to replace
        word_to_replace, _ = random.choice(replaceable_words)
        synonyms = get_synonyms(word_to_replace)

        if synonyms:
            # Replace with a random synonym
            new_word = random.choice(synonyms)
            altered_sentence = altered_sentence.replace(word_to_replace, new_word, 1)
            # Remove the replaced word from the list to avoid replacing it again
            replaceable_words = [(word, tag) for word, tag in replaceable_words if word != word_to_replace]

    print("sentence")
    print(sentence)
    print(altered_sentence)

    return altered_sentence

### some examples:

Charlton , 61 , and his wife , Peggy , became citizens of Ireland when they formally received Irish passports from deputy Prime Minister Dick Spring who said the honour had been made in recognition of Charlton 's achievements as the national soccer manager .

--->

Charlton , 61 , and his wife , Peggy , became citizens of Ireland when they formally received Irish passports from deputy Prime Minister Dick Spring who said the honour had been made in recognition of Charlton 's achievements as the home soccer manager .



....


Charlton managed Ireland for 93 matches , during which time they lost only 17 times in almost 10 years until he resigned in December 1995 .

--->


Charlton managed Ireland for 93 matches , during which time they lost only 17 times in almost 10 years until he quit in December 1995 .

## Data Augmentation for back translation and synonym replacement

first, we need to create the directory in which we want to store the back_translated result.

In [ ]:
%mkdir new_dataset

In [ ]:
def augment_file_with_back_translation(file_path: str, target_language: str, output_file_path: str):
    sentences = []
    annotations = []

    # Read the file and collect sentences and annotations
    with open(file_path, 'r') as file:
        sentence = []
        annotation = []
        for line in file:
            if line.strip() == '':
                if sentence:
                    sentences.append(' '.join(sentence))
                    annotations.append(annotation)
                    sentence = []
                    annotation = []
                continue
            parts = line.strip().split()
            sentence.append(parts[0])
            annotation.append(parts[1:])

    # Perform back translation on each sentence
    # augmented_sentences = [back_translate_sentence(sentence, target_language) for sentence in sentences]

    # Perform synonym replacement on each sentence
    augmented_sentences = [replace_random_words_with_synonyms(sentence) for sentence in sentences]

    # Write the augmented sentences with the original annotations back to a new file
    with open(output_file_path, 'w') as outfile:
        for aug_sentence, annotation in zip(augmented_sentences, annotations):
            # Assuming back-translation does not change the number of tokens
            aug_tokens = aug_sentence.split()
            for token, ann in zip(aug_tokens, annotation):
                outfile.write(f'{token} {" ".join(ann)}\n')
            outfile.write('\n')

In [ ]:
translate_url = 'https://translation.googleapis.com/language/translate/v2'
api_key = 'YOUR_API_KEY'

def augment_dataset(name):
  file_path = './' + name + '.txt'  # Path to the original dataset file
  output_file_path = './synonym_replacement_4_' + name + '.txt'  # Path to save the augmented dataset file
  target_language = 'fa'  # Target language for translation, e.g., French
  augment_file_with_back_translation(file_path, target_language, output_file_path)

augment_dataset('wiki_train')
# augment_dataset('train')
# augment_dataset('valid')
# augment_dataset('test')

sentence
April
April
sentence
April
April
sentence
April's
April's
sentence
The
The
sentence
April
April
sentence
April
April
sentence
In
atomic number 49
sentence
In
inwards
sentence
April
April
sentence
It
IT
sentence
Quite
quite an
sentence
April Poets
April poet
sentence
Events
event
sentence
Fixed
desexualise
sentence
April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April April
April April April April April April April April April April April April April April April April April April April April April April April April April April April Apr

## Entity Swap

In this section, we attemp to change the entities according to their entity type. We first analyzed the train file and made a dictionary, in which its keys are entity types and its values are words used in the training set as those keys' values. Then by iterating over file, we attemp to change entities with a random value associated with the specific key from the extracted dictionary. And finally save the results in a new file.

### here are some examples of changed entitis:


LEICESTERSHIRE NNP B-NP B-ORG
Uniceramic NNP B-NP B-ORG

...

LONDON NNP B-NP B-LOC
Waterville NNP B-NP B-LOC

...

Phil NNP I-NP B-PER
Sadiki NNP I-NP B-PER

...

Simmons NNP I-NP I-PER
Zwingmann NNP I-NP I-PER

...

Leicestershire NNP B-NP B-ORG
Kredietbank NNP B-NP B-ORG

...

Somerset NNP B-NP B-ORG
LKS NNP B-NP B-ORG

In [ ]:
!wget 'https://drive.google.com/uc?id=19A400XFYJTZL_P3HXjCEIEnDdMTDYbzk' -O extracted_entities_train.txt

--2024-02-23 12:27:58--  https://drive.google.com/uc?id=19A400XFYJTZL_P3HXjCEIEnDdMTDYbzk
Resolving drive.google.com (drive.google.com)... 74.125.135.100, 74.125.135.102, 74.125.135.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.135.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=19A400XFYJTZL_P3HXjCEIEnDdMTDYbzk [following]
--2024-02-23 12:27:58--  https://drive.usercontent.google.com/download?id=19A400XFYJTZL_P3HXjCEIEnDdMTDYbzk
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.202.132, 2607:f8b0:400e:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.202.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68118 (67K) [application/octet-stream]
Saving to: ‘extracted_entities_train.txt’

extracted_entities_ 100%[===================>]  66.52K  --.-KB/s    in 0.001s  

2024-02-23 12:27:5

In [ ]:
import random

def load_entities_from_file(file_path):
    entities = {}
    current_type = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            if line.endswith(':'):
                current_type = line[:-1]
                entities[current_type] = []
            else:
                entities[current_type].append(line)

    return entities

def entity_swap(file_path, output_file_path):
    # Open and read the input file line by line
    with open(file_path, 'r') as file:
        # Open the output file for writing
        with open(output_file_path, 'w') as output_file:
            for line in file:
                line = line.strip()
                print(line)
                # Check if the line is not empty
                if line:
                    # Split the line by space to extract the last word
                    words = line.split()
                    last_word = words[-1]
                    # Check if the first word is a key in the loaded_entities dictionary
                    if last_word in loaded_entities:
                        # Replace the first word with a random value from the dictionary
                        random_value = random.choice(loaded_entities[last_word])
                        # Replace the first word with the random value
                        words[0] = random_value
                        # Join the words back into a line
                        new_line = ' '.join(words)
                        output_file.write(new_line + '\n')
                    else:
                        # If the first word is not in the dictionary, write the original line to the output file
                        output_file.write(line + '\n')
                else:
                    # If the first word is not in the dictionary, write the original line to the output file
                    output_file.write(line + '\n')


In [ ]:


# Load entities from the dictionary
entities_file_path = './extracted_entities_train.txt'
loaded_entities = load_entities_from_file(entities_file_path)

# Loop through the keys and concatenate the values for each key into a single sentence
for key in loaded_entities:
    values = loaded_entities[key]
    concatenated_values = ' '.join(values)
    print(f"Key: {key}")
    print("Concatenated Values:")
    print(concatenated_values)
    print()


Key: B-ORG
Concatenated Values:
Barrick Daily Trade U.S. Arsenal Correction China Itar-Tass AdOn Earth Husqvarna Ujpest CVRD MX Bayern DINAMO Eli Coast Polonia Kent West Duma Tromso France-Soir ZTE Hwa Pasok Napoli Ajax Zaglebie Palermo Ipswich Advanced Tekstilshik Hemina Empoli BARCELONA BOLAND Collingwood UEFA Hajduk Lenzing Southend Banfield Kpaitan Scac Politehnica Eletropaulo Malbak NEUCHATEL Bari IRNA Istanbul LE Ghent Pentagon RSA Rovers Torino Auxerre Checker Bailard Impala Repsol Aki VNU Tottenham Hibernian Csepel MIT Royal Start Pakistan Internet Charlton Racing France Glentoran Richmond Hampshire Scunthorpe DBRS Bestop Karabach Roma wall Footscray MANCHESTER Cagliari Raja Metro ICAC Notts Airdrieonians Hanson Expos M&R New Rugby Gente PULPEX Congress Lillestrom Triple-A Leverkusen Tenacity Union Brush UBS KERDOS Agriculture Globo Sloga RUF Boeing SEC METROSTAV RBA Lufthanseat Jiul LKS Loxley Toyota Husaberg Greenville Springbok Spartak Anyang Vicenza Kekkila F.C. Escravos Su

In [ ]:
# Define the file path
# file_path = 'test.txt'
# output_file_path = 'entity_swapped_test.txt'


# entity_swap('test.txt', 'entity_swapped_test.txt')
# entity_swap('train.txt', 'entity_swapped_train.txt')
# entity_swap('valid.txt', 'entity_swapped_valid.txt')
entity_swap('wiki_train.txt', 'entity_swapped_train.txt')

April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.

April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.

April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.

The Month

April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.

April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.

In common years, April starts on the same day of the week as October of the previous year, and in leap years, Ma

### concating old and new files

In [ ]:


def concatFiles(method):

  data_splits = ["wiki_train"]
  # data_splits = ["test", "train", "valid"]
  for i in range(len(data_splits)):
    # Define the file paths
    file1_path = data_splits[i] + '.txt'
    file2_path = method + '_' + data_splits[i] + '.txt'
    output_file_path = 'concat_original_with_' + file2_path
    # Open the first file and write its content to the output file
    with open(file1_path, 'r') as file1, open(output_file_path, 'w') as output_file:
        output_file.writelines(file1.readlines())
    # Open the second file, skip the first line, and write the rest to the output file
    with open(file2_path, 'r') as file2, open(output_file_path, 'a') as output_file:
        next(file2)  # Skip the first line
        output_file.writelines(file2.readlines())
    print("Files concatenated successfully.")


concatFiles('synonym_replacement_2')
concatFiles('synonym_replacement_3')
concatFiles('entity_swapped')

Files concatenated successfully.
Files concatenated successfully.
Files concatenated successfully.


In [ ]:
data_splits = ["test", "train", "valid"]
data_splits[0]

'test'